In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
print(x.get_shape())
y = tf.placeholder("float", [None, n_classes])
print(y.get_shape())
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

(?, 28, 28)
(?, 10)


In [6]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    print(x.get_shape())
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    print(x.get_shape())
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    # x = tf.split(x, n_steps, 0)
    x = tf.split(0, n_steps, x)
    print(len(x))
    print(x[0].get_shape())
    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = tf.nn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

(28, ?, 28)
(?, 28)
28
(?, 28)


In [7]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.687786, Training Accuracy= 0.42188
Iter 2560, Minibatch Loss= 1.384425, Training Accuracy= 0.58594
Iter 3840, Minibatch Loss= 1.062889, Training Accuracy= 0.65625
Iter 5120, Minibatch Loss= 0.908316, Training Accuracy= 0.72656
Iter 6400, Minibatch Loss= 0.764098, Training Accuracy= 0.77344
Iter 7680, Minibatch Loss= 1.044616, Training Accuracy= 0.62500
Iter 8960, Minibatch Loss= 0.816776, Training Accuracy= 0.72656
Iter 10240, Minibatch Loss= 0.550322, Training Accuracy= 0.83594
Iter 11520, Minibatch Loss= 0.379904, Training Accuracy= 0.89062
Iter 12800, Minibatch Loss= 0.690672, Training Accuracy= 0.77344
Iter 14080, Minibatch Loss= 0.449672, Training Accuracy= 0.86719
Iter 15360, Minibatch Loss= 0.345327, Training Accuracy= 0.88281
Iter 16640, Minibatch Loss= 0.436947, Training Accuracy= 0.90625
Iter 17920, Minibatch Loss= 0.304931, Training Accuracy= 0.90625
Iter 19200, Minibatch Loss= 0.258700, Training Accuracy= 0.91406
Iter 20480, Minibatch Loss= 0.16

In [8]:
batch_x, batch_y = mnist.train.next_batch(batch_size)
print(batch_x.shape)
batch_x = batch_x.reshape((batch_size, n_steps, n_input))
print(batch_x.shape)

(128, 784)
(128, 28, 28)


In [11]:
print(mnist.test.labels[0])

[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
